In [ ]:
import numpy as np
import sympy as sp

In [ ]:
def criar_vetor_de_simbolos(tamanho):
    """
    Cria um vetor (lista) de símbolos SymPy com o tamanho especificado.

    Parameters:
    tamanho (int): O número de símbolos a serem criados.

    Returns:
    list: Lista contendo os símbolos criados.
    """
    # Cria os símbolos usando sp.symbols
    return sp.symbols(f'x1:{tamanho+1}')

In [ ]:
sp.Matrix(np.array_split(criar_vetor_de_simbolos(4), 2))

In [ ]:
# Função para calcular o produto Kronecker considerando binário exclusivo
def kronecker_binary(vectors):
    result = np.array([1])  # Inicia com o elemento neutro da multiplicação
    for v in vectors:
        result = np.kron(result, v)
    return result

In [ ]:
kronecker_binary([[1,0],[0,1]])

In [ ]:
bits = 2
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

In [ ]:
bits = 4
variaveis = sp.Matrix(kronecker_binary(np.array_split(criar_vetor_de_simbolos(bits*2), bits)))
variaveis

In [ ]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(variaveis):
    print(f"Linha {linha}: {elemento}")

In [ ]:
def decimal_para_binario_vetor(numero_decimal, tamanho_bits=8):
    """
    Converte um número decimal para um vetor binário usando produtos de Kronecker.

    Args:
        numero_decimal (int): Número decimal a ser convertido.
        tamanho_bits (int, opcional): Número de bits para representar o número binário. Padrão é 8.

    Returns:
        np.ndarray: Vetor resultante da conversão.
    
    Raises:
        ValueError: Se o número decimal for negativo ou se o tamanho de bits for insuficiente.
    """
    # Mapeamento dos caracteres binários para os vetores desejados
    mapeamento = {'0': [1, 0], '1': [0, 1]}

    # Verifica se o número é um inteiro não negativo
    if not isinstance(numero_decimal, int):
        raise TypeError("O número decimal deve ser um inteiro.")
    if numero_decimal < 0:
        raise ValueError("A função não suporta números decimais negativos.")

    # Converter o número decimal para binário e remover o prefixo '0b'
    numero_binario = bin(numero_decimal)[2:]

    # Verifica se o número binário cabe no tamanho de bits desejado
    if len(numero_binario) > tamanho_bits:
        raise ValueError(f"O número binário '{numero_binario}' excede o tamanho de bits especificado ({tamanho_bits} bits).")

    # Adiciona zeros à esquerda para completar o tamanho de bits desejado
    numero_binario = numero_binario.zfill(tamanho_bits)

    # Inicializa o vetor com 1 para o produto de Kronecker
    vetor = np.array([1])
    
    # Aplica o produto de Kronecker para cada dígito binário
    for digito in numero_binario:
        vetor = np.kron(vetor, mapeamento[digito])

    return vetor

In [ ]:
saida_tabela = [
    [0, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 0],
    [1, 0, 1, 1, 0, 1, 1],
    [1, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 1, 1],
    [1, 1, 0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 1, 1, 0, 0],
    [0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0],
    [1, 1, 1, 1, 0, 0, 1],
    [1, 1, 1, 0, 0, 0, 1]
]

#saida_tabela = [linha[::-1] for linha in saida_tabela]

vetores_binarios = []

for comb in saida_tabela:
	vetor = kronecker_binary(np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]))
	vetores_binarios.append(vetor)

vetores_binarios = np.array(vetores_binarios).T
sp.Matrix(vetores_binarios)

In [ ]:
vetores_binarios.tolist()

In [ ]:
from sympy import symbols
from sympy.logic.boolalg import And, Or, Not, Xor
from sympy import simplify_logic

def matrix_to_boolean_function_general(matrix):
    """
    Converte uma matriz binária (0/1) de transformação linear discreta
    em expressões booleanas para cada bit de saída, com qualquer número
    de bits de entrada e saída (matriz pode ser retangular).
    """
    n_outputs = len(matrix)       # número de linhas
    n_inputs = len(matrix[0])     # número de colunas

    # Calcula quantos bits são necessários para codificar entradas e saídas
    input_bits_count = (n_inputs - 1).bit_length()
    output_bits_count = (n_outputs - 1).bit_length()

    # Gera variáveis simbólicas para os bits de entrada
    input_symbols = symbols(' '.join(f'x{i}' for i in range(input_bits_count)))

    def index_to_bits(i, width):
        return tuple((i >> k) & 1 for k in reversed(range(width)))

    output_expressions = []

    for output_bit in range(output_bits_count):
        minterms = []

        for col in range(n_inputs):
            col_vector = [matrix[row][col] for row in range(n_outputs)]
            if any(col_vector):
                output_vector_index = col_vector.index(1)
                output_bits = index_to_bits(output_vector_index, output_bits_count)
                if output_bits[output_bit] == 1:
                    input_bits = index_to_bits(col, input_bits_count)
                    term = []
                    for idx, val in enumerate(input_bits):
                        symbol = input_symbols[idx]
                        term.append(symbol if val else Not(symbol))
                    minterms.append(And(*term))

        if minterms:
            output_expr = simplify_logic(Or(*minterms), form='dnf')
        else:
            output_expr = 0

        output_expressions.append((f'bit {output_bit}', output_expr))

    return input_symbols, output_expressions


input_symbols, exprs_general = matrix_to_boolean_function_general(vetores_binarios)

In [ ]:
def generate_function_from_exprs_general(exprs_with_vars, fn_name="logic_fn"):
    """
    Gera código Python para uma função lógica baseada em expressões booleanas
    e símbolos de entrada arbitrários (não limitado a a, b).
    """
    input_symbols, exprs_text = exprs_with_vars
    param_list = ", ".join(str(sym) for sym in input_symbols)

    # Construir o corpo da função com cada bit de saída
    lines = []
    for i, (bit_name, expr) in enumerate(exprs_text):
        lines.append(f"    bit{i} = {expr}")
    lines.append(f"    return ({', '.join([f'bit{i}' for i in range(len(exprs_text))])})")

    # Construir a função completa como string
    fn_code = f"def {fn_name}({param_list}):\n" + "\n".join(lines)
    return fn_code

# Aplicar ao exemplo anterior com 2 bits de entrada → 3 bits de saída
generated_code_general = generate_function_from_exprs_general((input_symbols, exprs_general), fn_name="bit_expander")
print(generated_code_general)

In [ ]:
def bit_expander(x0, x1, x2, x3):
    bit0 = (x0 & x3) | (x0 & ~x1) | (x2 & ~x1) | (x2 & ~x3) | (x1 & ~x0 & ~x2)
    bit1 = (x0 & x2) | (x0 & ~x1) | (x1 & ~x3) | (~x2 & ~x3) | (x1 & ~x0 & ~x2)
    bit2 = (x0 & x1) | (x0 & x2) | (x2 & ~x3) | (~x1 & ~x3)
    bit3 = (x0 & ~x2) | (x1 & x2 & ~x3) | (x1 & x3 & ~x2) | (x2 & x3 & ~x1) | (~x0 & ~x1 & ~x3)
    bit4 = (x0 & ~x1) | (x3 & ~x0) | (x3 & ~x2) | (~x1 & ~x2) | (x1 & x2 & ~x0)
    bit5 = (~x0 & ~x1) | (~x1 & ~x3) | (x0 & x3 & ~x2) | (x2 & x3 & ~x0) | (~x0 & ~x2 & ~x3)
    bit6 = ~x3 | (x1 & x2) | (x1 & ~x0) | (x2 & ~x0) | (x0 & ~x1 & ~x2)
    return (bit0, bit1, bit2, bit3, bit4, bit5, bit6)

In [ ]:
custom_logic(0,0,0,1)

In [ ]:
# Lista para armazenar os índices de cada linha
resultado = []

# Percorre cada linha da matriz
for linha in vetores_binarios:
    indices = [i for i, val in enumerate(linha) if val == 1]
    resultado.append(indices if indices else [None])
    
resultado

In [ ]:
regra = sp.Matrix(np.dot(vetores_binarios, variaveis)) # expressões simbólicas
regra

In [ ]:
regra.shape

In [ ]:
# Iterando pelo vetor com o índice e o valor
for linha, elemento in enumerate(regra):
    print(f"Linha {linha}: {elemento}")

In [ ]:
lista = regra.tolist()[0]
list(set([x for x in lista if lista.count(x) > 1]))

In [ ]:
#Sumulando saidas
teste = np.dot(vetores_binarios, decimal_para_binario_vetor(0, 4))
teste

In [ ]:
teste.size

In [ ]:
[i for i, val in enumerate(teste) if val == 1]

In [ ]:
# Lista para armazenar os índices de cada linha
resultado = []

# Percorre cada linha da matriz
for linha in vetores_binarios:
    indices = [i for i, val in enumerate(linha) if val == 1]
    resultado.append(indices if indices else [None])
    
resultado

In [ ]:
from sympy.utilities.lambdify import lambdastr

#  Lista para armazenar as funções
funcoes = []

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

# Iterar sobre cada elemento do vetor simbólico e criar uma função
for expr in regra:
    func = sp.lambdify(variaveis, expr)
    funcoes.append(func)


In [ ]:
from sympy.utilities.lambdify import lambdastr

# Variáveis que as funções irão receber
variaveis = criar_vetor_de_simbolos(bits*2)

for expr in regra:
    print(lambdastr(variaveis, expr))

In [ ]:
import itertools

# Usando produto cartesiano
combinacoes = list(itertools.product([0, 1], repeat=4))

for comb in combinacoes:
	# Converte cada bit da combinação para um vetor binário e achata em uma única dimensão
	valores = np.array([decimal_para_binario_vetor(bit, 1) for bit in comb]).flatten()
	print(valores,[f(*valores) for f in funcoes])

In [ ]:
list(itertools.product([0, 1], repeat=7))

In [ ]:
"\overline{ab}cd+\overline{ab}c\overline{d}+ab\overline{c}d+\overline{a}b\overline{cd}+\overline{a}b\overline{c}d+a\overline{b}\overline{c}d+abcd+a\overline{b}c\overline{d}+abc\overline{d}+a\overline{b}cd+\overline{a}bc\overline{d}+a\overline{bcd}"

"https://www.boolean-algebra.com/?q=YWJ7Y2R9K3thYmNkfSt7YWJ9Y3tkfSthYntjfWQrYWJjZCthe2J9Y3tkfSthYmN7ZH0rYXtifWNkK3thfWJje2R9K2F7Yn17Y2R9"

"https://www.boolean-algebra.com/?q=YWJ7Y2R9K3thYmNkfSt7YWJ9Y2Qre2FifWN7ZH0rYWJ7Y31kK3thfWJ7Y31kK2F7Yn17Y31kK2FiY3tkfSthe2J9Y2Qre2F9YmN7ZH0rYXtifXtjZH0="

"https://www.boolean-algebra.com/?q=e2FifXtjfWQre2F9YmNkK3thYmNkfSt7YWJ9Y2QrYWJ7Y31kK3thfWJ7Y31kK2F7Yn17Y31kK2F7Yn1je2R9K2F7Yn1jZCt7YX1iY3tkfSthe2J9e2NkfQ=="